In [ ]:
import transformers
import pandas as pd
import torch

model_id = "hfl/llama-3-chinese-8b-instruct-v2"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

In [ ]:
def prompt_forming_zero(instruction, passage):
  messages = [
    {"role": "system", "content": "你是一個能幹的閱讀理解問題生成器，始終遵循給定的說明和要求來生成問題。"},
    {"role": "user", "content": f"{instruction}\n文章:{passage}"},
]
  prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)
  return prompt

def prompt_forming_few(instruction, example, passage):
  messages = [
    {"role": "system", "content": "你是一個能幹的閱讀理解問題生成器，始終遵循給定的說明和要求來生成問題。"},
    {"role": "user", "content": f"{instruction}\n{example}\n文章:{passage}"},
]
  prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)
  return prompt

def bot(prompt):
  terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]
  print(prompt)
  outputs = pipeline(
      prompt,
      max_new_tokens=200,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
      )
  print(outputs[0]["generated_text"][len(prompt):])
  return outputs[0]["generated_text"][len(prompt):]


def generate_question(article, prompt_list, level, way):
    result_list = []
    if level == 1:
      instruction = prompt_list[0]
      #due to copyright, we are unable to display the article content
      example = """範例文章及相應的範例問題(請參考範例來創作問題):{範例文章:{}
PIRLS第一層次範例問題1:{開學第一天老師做了什麼?
答案: 介紹自己}

PIRLS第一層次範例問題2:{小息時學生的狀態怎麼樣？
答案: 精神興奮}

PIRLS第一層次範例問題3:{文中惡霸能屢屢得手的原因是什麼？
答案: 弱者不敢告發他們}

PIRLS第一層次範例問題4:{惡霸大威受到什麼懲罰？
答案: 被老師狠狠教訓一番}

PIRLS第一層次範例問題5:{文中校園惡霸什麼形象？
答案: 身體健碩}}"""

    elif level == 2:
      instruction = prompt_list[1]
      example = """範例文章及相應的範例問題(請參考範例來創作問題):{範例文章:{}
PIRLS第二層次範例問題1:{莉莉最好的朋友是谁
答案: 鲍思高}

PIRLS第二層次範例問題2:{为什么说莉莉家很有钱？
答案: 因为她的爸爸是一个成功的商人}

PIRLS第二層次範例問題3:{为什么鲍勃没有从他的农田里赚到钱
答案: 因为今年有很严重的飓风}

PIRLS第二層次範例問題4:{为什么莉莉让她的妈妈把她们的食物、衣服和玩具送给鲍勃？
答案: 因为她想要帮助有需要的人/因为她认为鲍勃一家比她更需要这些东西/因为她为鲍勃的遭遇感到难过}

PIRLS第二層次範例問題5:{为什么莉莉不再浪费食物了？
答案: 因为她知道了食物的价值}}"""

    elif level == 3:
      instruction = prompt_list[2]
      example = """範例文章及相應的範例問題(請參考範例來創作問題):{範例文章:{}
PIRLS第三層次範例問題1:{脑先生和手小姐的性格是什么？
答案: 知错能改}

PIRLS第三層次範例問題2:{脑先生和手小姐吵架后，主人有什么感受？
答案: 既苦恼又肚子饿}

PIRLS第三層次範例問題3:{脑先生和手小姐存在是为了什么？
答案: 为主人服务}

PIRLS第三層次範例問題4:{脑先生为什么觉得自己较厉害？
答案: 因为他觉得自己负责控制手小姐活动}

PIRLS第三層次範例問題5:{手小姐为什么觉得自己较厉害？
答案: 她认为没有她执行动作，脑先生什么也做不了，只能空想。}}"""

    elif level == 4:
      instruction = prompt_list[3]
      example = """範例文章及相應的範例問題(請參考範例來創作問題):{範例文章:{}
PIRLS第四層次範例問題1:{我们可以怎么形容机器人？
答案: 很有用}

PIRLS第四層次範例問題2:{作者发现房中的一切之后有什么感觉？
答案: 震惊}

PIRLS第四層次範例問題3:{你会怎么形容作者没有清理前的房间？
答案: 震耳欲聋/吵闹/杂乱}

PIRLS第四層次範例問題4:{这个故事教会了我们什么？
答案: 我们应该要保持清洁和干净}

PIRLS第四層次範例問題5:{你觉得最后作者的妈妈会感觉怎么样？
答案: 她对作者很骄傲/她觉得作者做的很棒/她很开心作者终于对自己的房间负责任}}"""

    if way == "zero":
      prompt = prompt_forming_zero(instruction, article)
    elif way == "few":
      prompt = prompt_forming_few(instruction, example, article)
    output = bot(prompt=prompt)
    result_list.append([article, output, level])
    return result_list

def process(article_data,prompt_path, way):
      prompt_list = []
      temp_prompt_data = pd.read_excel(prompt_path)
      for n in range(len(temp_prompt_data)):
          prompt_list.append(temp_prompt_data['prompt'][n])
      result_list = []
      for n in range(len(article_data)):
          print(f"{n+1}/50:")
          article = article_data['clean_article'][n]
          result_list.extend(generate_question(article, prompt_list, level=1, way=way))
          result_list.extend(generate_question(article, prompt_list, level=2, way=way))
          result_list.extend(generate_question(article, prompt_list, level=3, way=way))
          result_list.extend(generate_question(article, prompt_list, level=4, way=way))
      df = pd.DataFrame(result_list, columns = ['article', 'question', 'required level'])
      if way == "zero":
          df.to_excel(f"zero.xlsx", index=None)
      elif way == "few":
          df.to_excel(f"few.xlsx", index=None)

article_data = pd.read_excel("test article.xlsx")
prompt_path = "QAG_prompt_zero.xlsx"
way = "zero"
process(article_data, prompt_path, way)